Results not good

In [9]:
import PyPDF2


In [10]:
def extract_text_from_pdf(pdf_path):
    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as pdf_file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        
        # Initialize an empty string to store extracted text
        text = ""
        
        # Iterate through each page of the PDF
        for page_num in range(len(pdf_reader.pages)):
            # Get a specific page from the PDF
            page = pdf_reader.pages[page_num]
            
            # Extract text from the page
            page_text = page.extract_text()
            
            # Append the extracted text to the overall text string
            text += page_text
            
        # Return the extracted text
        return text

In [11]:
pdf_path = '/Users/princypatel/valunova/bill copy valunova jan.pdf'  # Replace 'example.pdf' with the path to your PDF file
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

RANJAN MEDICAL
PURANI KOTWALI  ASHIFGANJ, 
AZAMGARH, UTTAR PRADESH - 276001
Page : 1 Of 1 9838586345, 9454979291
Bill Copy, Bill By Product(Party Wise) From : 01/01/24 To 31/01/24
INV NO      INV DATE  PARTY / PRODUCT                    QTY     FREE    TOTAL AMT     SCHEME
                                                                                              
                      ASWINI HEALTHCARE                                                       
                      GST:, Mob:                                                              
SALE-L3475  24/01/24  PENCENAX-MR TABS                    10        0       341.67        2+1 
                      CASH SALES                                                              
                      GST:, Mob:                                                              
SALE-L3384  15/01/24  ROVAFLUX DSR CAPS                    2        0        65.33        1+2 
SALE-L3424  18/01/24  INFIZITH-500 TAB                    20 

In [12]:
import csv

In [13]:
def save_text_to_csv(extracted_text, csv_file):
    # Open the CSV file in append mode
    with open(csv_file, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Loop through each page's text
        for page_text in extracted_text:
            lines = page_text.split('\n')
            data_start = False
            
            # Iterate through each line
            for line in lines:
                # Check if the line contains the start of the data section
                if "INV NO      INV DATE  PARTY / PRODUCT" in line:
                    data_start = True
                    continue
                
                # If the data section has started, extract and write data to CSV
                if data_start:
                    # Extracting data based on fixed positions
                    invoice_no = line[:11].strip()
                    invoice_date = line[11:21].strip()
                    party_product = line[21:53].strip()
                    quantity = line[53:61].strip()
                    free = line[61:69].strip()
                    total_amt = line[69:84].strip()
                    scheme = line[84:].strip()
                    
                    # Write data to CSV
                    writer.writerow([invoice_no, invoice_date, party_product, quantity, free, total_amt, scheme])


In [14]:
# Path to save CSV file
csv_file = '/Users/princypatel/valunova/bill_copy_valunova_jan.csv'

# Save extracted text to CSV
save_text_to_csv(extracted_text, csv_file)


In [15]:
import tabula 

In [16]:
data= tabula.read_pdf('/Users/princypatel/valunova/bill copy valunova jan.pdf', pages= '1')

/Users/princypatel/valunova/venv/lib/python3.10/site-packages/tabula/io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [17]:
print(data)

[                                      Unnamed: 0  Unnamed: 1  \
0                INV NOINV DATE  PARTY / PRODUCT         NaN   
1                                            NaN         NaN   
2                              ASWINI HEALTHCARE         NaN   
3                                     GST:, Mob:         NaN   
4         SALE-L3475  24/01/24  PENCENAX-MR TABS         NaN   
5                                     CASH SALES         NaN   
6                                     GST:, Mob:         NaN   
7        SALE-L3384  15/01/24  ROVAFLUX DSR CAPS         NaN   
8         SALE-L3424  18/01/24  INFIZITH-500 TAB         NaN   
9      SALE-L3520  31/01/24  FIXTERIA-CV 200 TAB         NaN   
10                                   PARTY TOTAL         NaN   
11                                     DR S LARI         NaN   
12                                    GST:, Mob:         NaN   
13       SALE-L3401  16/01/24  FIXTERIA 200 TABS         NaN   
14                              DR TANV

In [4]:
import pdfplumber
import pandas as pd
import numpy as np

# Path to the PDF file
pdf_file = '/Users/princypatel/valunova/bill copy valunova jan.pdf'

# Function to parse dates
def parse_date(date_str):
    try:
        # Attempt to parse the date string
        return pd.to_datetime(date_str)
    except:
        # If parsing fails, return NaN
        return np.nan

# Function to extract data from PDF
def extract_data_from_pdf(pdf_file):
    data = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            lines = text.split('\n')
            for line in lines:
                # Split line into individual fields based on the fixed positions
                inv_no = line[:11].strip()
                inv_date = line[11:21].strip()
                party_product = line[21:53].strip()
                qty = line[53:61].strip()
                free = line[61:69].strip()
                total_amt = line[69:84].strip()
                scheme = line[84:].strip()
                
                # Append extracted data to the list
                data.append([inv_no, inv_date, party_product, qty, free, total_amt, scheme])
    return data

# Extract data from PDF
data = extract_data_from_pdf(pdf_file)

# Define column names for DataFrame
columns = ['INV NO', 'INV DATE', 'PARTY / PRODUCT', 'QTY', 'FREE', 'TOTAL AMT', 'SCHEME']

# Create DataFrame
df = pd.DataFrame(data, columns=columns)

# Replace empty strings with NaN
df.replace('', np.nan, inplace=True)

# Parse 'INV DATE' column
df['INV DATE'] = df['INV DATE'].apply(parse_date)

# Convert numeric columns to appropriate data types
numeric_cols = ['QTY', 'FREE', 'TOTAL AMT']
df[numeric_cols] = df[numeric_cols].astype(float)

# Save DataFrame to CSV
csv_file = 'invoices2.csv'
df.to_csv(csv_file, index=False)



/var/folders/f1/j3g41jpn45lgzzz9dzbxrdn40000gn/T/ipykernel_25157/3289693552.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('', np.nan, inplace=True)


ValueError: could not convert string to float: '4 To 31/'